In [ ]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt

In [ ]:
# Load your CSV data
movement_data = pd.read_csv("../data/movements.csv", parse_dates=["datetime"])  # Replace with your file path
restaurant_data = pd.read_csv("../data/restaurants.csv")  # Replace with your file path

# Clean Data

In [ ]:
movement_data.columns = movement_data.columns.str.lower()
restaurant_data.columns = restaurant_data.columns.str.lower()
restaurant_data.rename(columns={'restaurant id': 'restaurant_id'}, inplace=True)

print(movement_data.head())
print(restaurant_data.head())



In [ ]:
print(movement_data.isnull().sum())
print(restaurant_data.isnull().sum())

# Exploratory Data Analysis

## How many people

In [ ]:
print(len(movement_data.groupby('id')))

## Entries per person

In [ ]:
print(movement_data.groupby('id')['datetime'].count().head(10))

## Graphs of People's Centeral Location

### Haverstine function

In [ ]:
# Haversine distance function (Earth radius in kilometers)
def haversine(lon1, lat1, lon2, lat2):
    R = 6371  # Radius of Earth in kilometers
    lon1, lat1, lon2, lat2 = map(np.radians, [lon1, lat1, lon2, lat2])
    dlon = lon2 - lon1
    dlat = lat2 - lat1
    a = np.sin(dlat/2)**2 + np.cos(lat1) * np.cos(lat2) * np.sin(dlon/2)**2
    c = 2 * np.arcsin(np.sqrt(a))
    return R * c

### Graphs

In [ ]:
centroid = movement_data.groupby('id').agg({'longitude': 'mean', 'latitude': 'mean'}).reset_index()
centroid.columns = ['id', 'centroid_longitude', 'centroid_latitude']

In [ ]:
# Create scatter plot
plt.figure(figsize=(8, 5)) 
plt.scatter(centroid['centroid_longitude'], centroid['centroid_latitude'],label='people', color='blue', marker='o', s=100, alpha=0.7)

# Add title and labels
plt.title('Centeroid Points of People')
plt.xlabel('Longitude')
plt.ylabel('Latitude')

# Show grid
plt.grid(True)

# Display the plot
plt.show()

In [ ]:
plt.scatter(centroid['centroid_longitude'], centroid['centroid_latitude'],label='People', color='blue', marker='o', s=100, alpha=0.7)
plt.scatter(restaurant_data['longitude'], restaurant_data['latitude'],label='Restaurants', color='red', marker='s', s=100, alpha=0.7)

# Add title and labels
plt.title('Centroid Points of People and Restaurant Locations')
plt.xlabel('Longitude')
plt.ylabel('Latitude')
plt.legend()
plt.xticks(rotation=45)
plt.grid(True)
plt.show()


## Paths

In [ ]:
person1 = movement_data[movement_data['id'] == 'I000']
person1.head()

## Average Time Between Measurements

In [ ]:
movement_data['timestamp'] = pd.to_datetime(movement_data['datetime'])

# Calculate time differences
time_diffs = movement_data['timestamp'].diff()

# Compute the average time difference
average_time_diff = time_diffs.mean()

print(average_time_diff)

## People's Visits to Restaurants

In [ ]:


# Define a function to check if each movement is near a restaurant
def is_near_restaurant(movement, restaurants, threshold_km=0.5):
    # Ensure 'longitude' and 'latitude' exist in movement
    if 'longitude' not in movement or 'latitude' not in movement:
        raise ValueError("Movement must contain 'longitude' and 'latitude' keys.")

    # Iterate over the rows of the restaurants DataFrame
    for index, restaurant in restaurants.iterrows():
        # Calculate the distance using the haversine function
        distance = haversine(movement['longitude'], movement['latitude'],
                             restaurant['longitude'], restaurant['latitude'])
        
        # Check if the distance is within the threshold
        if distance <= threshold_km:
            return True, restaurant['restaurant_id']  # Return True and the restaurant ID

    return False, None  # Return False and None if no nearby restaurant is found


# Apply function to each movement and create a column indicating restaurant visits
# movement_data['restaurant_id'] = movement_data.apply(
#     lambda row: foo(count, row, restaurant_data), axis=1 
# )
for i in range(0, 100):
    person_id = movement_data.head(1)['id']
    print(f'{person_id}: {is_near_restaurant(movement_data.iloc[i], restaurant_data)[1]}')


# Loop through each row in the movement DataFrame
# for index, movement in movement_data.iterrows():
#     is_near, restaurant_id = is_near_restaurant(movement, restaurant_data)
#     print(f"Movement ID: {movement['id']} - Is near a restaurant? {is_near}, Restaurant ID: {restaurant_id}")